In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
model_outputs = pd.read_csv("data/final_model_output.csv")

In [45]:
# G2 VS Misfits game 1 LEC summer playoffs upper bracket semi finals
# model_outputs.query("league=='LEC' & date > '2022-01-01' & draft_diff > 0.20")
df = pd.DataFrame(model_outputs.iloc[33666]).transpose()

# G2 comp5 - MSF comp 3
# ('draft_agnostic_bot_lead_prob', 0.6001669664494229), ('post_draft_bot_lead_prob', 0.6358389650400617),
# ('draft_agnostic_mid_lead_prob', 0.6166685921758286), ('post_draft_mid_lead_prob', 0.635445881988775),
# ('draft_agnostic_top_lead_prob', 0.5609168819131201), ('post_draft_top_lead_prob', 0.6112936765169829),

# MSF: mid game comp with dive
# G2: late game comp
# G2 Picked a scaling comp which also increased their chances of winning lane.  

# G2:  Sejuani, Nocturne, Ahri, Yasuo, Senna
# MSF: Ornn, Vi, Sylas, Sivir, Lulu




| id | team_Dives | team_Tanks | team_Damages | team_Enchanters | team_Picks | team_Pokes | team_Engages | team_Splitpushs | early_game | mid_game | late_game | no_damage_type |
|---:|-----------:|-----------:|-------------:|----------------:|-----------:|-----------:|-------------:|----------------:|-----------:|---------:|----------:|---------------:|
|  0 |   0.548780 |   1.295122 |     1.079268 |        0.413415 |   0.201220 |   0.451220 |     0.806098 |        0.204878 |   1.113415 | 1.926829 |  1.959756 |            1.0 |
|  1 |   0.659113 |   0.645243 |     0.587810 |        0.180309 |   0.777105 |   1.308263 |     0.762649 |        0.079508 |   2.445399 | 1.444813 |  1.109787 |            0.0 |
|  2 |   0.522190 |   0.660606 |     1.313001 |        1.070381 |   0.293451 |   0.539785 |     0.561877 |        0.038710 |   1.488563 | 1.263539 |  2.247898 |            0.0 |
|  3 |   0.998295 |   0.694970 |     1.017391 |        0.096675 |   0.545780 |   0.462234 |     1.133333 |        0.051321 |   1.066837 | 3.207502 |  0.725661 |            0.0 |
|  4 |   0.327380 |   1.926278 |     1.241307 |        0.052252 |   0.286719 |   0.740072 |     0.415542 |        0.010450 |   0.836975 | 1.836215 |  2.326810 |            0.0 |
|  5 |   0.409686 |   0.502618 |     2.028959 |        0.012435 |   0.302029 |   0.250164 |     1.492965 |        0.001145 |   1.275687 | 1.659031 |  2.065281 |            0.0 |
|  6 |   0.248384 |   0.745330 |     1.085309 |        0.210848 |   0.389188 |   0.542205 |     0.741559 |        1.037177 |   1.000180 | 1.811243 |  2.188578 |            0.0 |

In [46]:
print(list(zip(df.columns, df.values[0])), sep='\n')

[('gameid', '2022-08-27 13:57:241839LEC'), ('teamname', 'G2 Esports'), ('opponent', 'Misfits Gaming'), ('date', '2022-08-27 13:57:24'), ('league', 'LEC'), ('result', 1), ('side', 1), ('elo_diff', 3.673384639961455), ('draft_agnostic_bot_lead_prob', 0.6001669664494229), ('draft_agnostic_mid_lead_prob', 0.6166685921758286), ('draft_agnostic_top_lead_prob', 0.5609168819131201), ('post_draft_bot_lead_prob', 0.6358389650400617), ('post_draft_mid_lead_prob', 0.635445881988775), ('post_draft_top_lead_prob', 0.6112936765169829), ('team_comp0_v_opp_comp0', 0), ('team_comp0_v_opp_comp1', 0), ('team_comp0_v_opp_comp2', 0), ('team_comp0_v_opp_comp3', 0), ('team_comp0_v_opp_comp4', 0), ('team_comp0_v_opp_comp5', 0), ('team_comp0_v_opp_comp6', 0), ('team_comp1_v_opp_comp0', 0), ('team_comp1_v_opp_comp1', 0), ('team_comp1_v_opp_comp2', 0), ('team_comp1_v_opp_comp3', 0), ('team_comp1_v_opp_comp4', 1), ('team_comp1_v_opp_comp5', 0), ('team_comp1_v_opp_comp6', 0), ('team_comp2_v_opp_comp0', 0), ('team_c